# HPE Container Platform API series  - Lab 2
## Launching a distributed application cluster for AI programmatically through REST API calls as a Big Data/AI tenant user.

**Requirements:**
- HPE Container Platform deployment
- IP address of FQDN of the HPE Container Platform's controller host
- a Big Data/AI-ML tenant user account   

**Utilities:**   
- cURL  
- Jupyter Notebook server with bash kernel installed

**Definitions:**
- *HPE Container Platform* is an enterprise-grade container platform designed to deploy both cloud-native and non-cloud-native applications whether on-premises, at the edge, in multiple public clouds, or in a hybrid model. This makes the HPE Container Platform ideal for helping application developers and data scientists accelerate their application development and deployment on **containers**, on-demand through a self-service portal and a RESTful API that surfaces programmable access. To learn more about HPE Container Platform visit the [HPE DEV portal](https://developer.hpe.com/platform/hpe-container-platform/home) and check out the blog articles.

- *tenant:* A tenant is a group of users created by the platform administrator. A tenant can represent for example, an office location, a business unit, an organization, a project, an application. A tenant is allocated a set of resources (CPU/GPU, RAM, Storage, App Store images, Kubernetes cluster) by the platform administrator. All the resources used by a tenant are not shared with other tenants. A tenant user is granted the role of member or admin for the tenant.


HPE Container Platform provides two types of tenants:  
* Big data, AI/ML tenants that exist within the context of HPE Container Platform. This type of tenants are also known as **EPIC** (Elastic Private Instant Cluster) tenants.
* Kubernetes tenants that exist within the context of one or more Kubernetes clusters managed by HPE Container Platform.
  
Here, in this lab part, I will cover how a big data, AI/ML (EPIC) tenant user can deploy, programmatically through REST API, distributed multi-node stateful applications for AI/ML and data analytics (for example TensorFlow, Spark, Hadoop) using pre-configured docker-based container images available in the HPE Container Platform Application Store. 

In EPIC mode, HPE Container Platform has embedded its own container orchestrator to support the unique requirements of non cloud-native, monolithic, distributed stateful applications (application state, storage, networking, resource management).

## The HPE Container Platform API Reference
The HPE Container Platform REST API allows you to achieve multiple actions programmatically, from performing administrative tasks, deploying cloud-native and non-cloud-native applications to scoring trained Machine Learning models.

Before you can call the HPE Container Platform API, you need to know what calls can be placed. The REST API reference documentation describes each object type, along with the supported operations, request input parameters, response model and response codes. 
To access the REST API reference documentation, obtain the IP address or hostname of the current active HPE Container Platform controller host from the administrator of the platform. Then in a web browser, navigate to the following URL:

``` 
http(s)://[Controller-IP-address-name]/apidocs
```

All the REST API calls are in the form: 
``` 
An HTTP VERB such as (GET , POST, DELETE, PUT, PATCH, UPDATE),  
A target API object: http(s)://[Controller-IP-address]:8080/api/v2/[object]
```

## Session Authentication in a multi-tenant environment
With the exception of some API calls, most of the REST API calls you can do against the HPE Container Platform API requires authentication. HPE Container Platform uses a *‘session location’* to use as operation context. In a multi-tenant environment, you request an authentication session location by issuing an authentication request in the following form:
* Call the API to request (POST) a new login session, providing username/password credentials as well as the Tenant name in the JSON body. The user must be a valid tenant user credentials with a role (member or admin) in the requested tenant. 
* Extract the resource path of the created *session location* object from the JSON response header,
* For each subsequent call, set a new HTTP Header with its key set to *X-BDS-SESSION* and its value set to the session location value and used as the *working tenant* context. 

``` 
Note: the session location will expire after 24 hours. 
```   

If you are not already familiar with REST API calls, I encourage you to check-out the [Understanding API basics and the value they provide](https://developer.hpe.com/blog/understanding-api-basics-and-the-value-they-provide) blog on HPE Developer Community portal. It explains you REST API concepts such as HTTP verbs you call against the API, the headers, and payloads used when making API calls. 

**cURL:** You will use cURL to make API requests. Information on cURL can be found [here](https://curl.haxx.se/)

#### Initialize the environment:

**IMPORTANT: Before running the next code cells, please make sure to adjust the login credentials environment variables below according to your tenant username and password**

In [1]:
#
# environment variables to be adjusted by the student
#
username="studentXX" # your tenant login credentials - username and password (it matches your Notebook Student account)
password="thepassword"
#
# fixed environment variables setup by the HPE CP administrator
#
controller_endpoint="controller.hpedevlab.net:8080"
tenantname="HackShack Tenant"
templateName="ttf"${username} #20 charcaters max
clusterName="ctf"${username} #20 characters max
echo $templateName
echo $clusterName

ttfstudent80
ctfstudent80


#### -1- Authenticate as Tenant user in the specified tenant:

In [2]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

This is your session location:  /api/v2/session/8d97047f-3120-454e-90fa-b8309a11b927
This is your session_Id: 8d97047f-3120-454e-90fa-b8309a11b927


#### -2- Make a quick check to ensure you can make REST API calls within your tenant working context:
Here you will fetch information about your session you have just established.

In [3]:
curl -k -s --request GET "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' | jq  #using jq to pretty print the JSON reponse of the API call 

{
  "_links": {
    "self": {
      "href": "/api/v2/session/8d97047f-3120-454e-90fa-b8309a11b927"
    },
    "all_sessions": {
      "href": "/api/v2/session"
    }
  },
  "user": "/api/v1/user/803",
  "user_name": "student80",
  "tenant": "/api/v1/tenant/3",
  "tenant_name": "HackShack Tenant",
  "role": "/api/v1/role/3",
  "role_name": "Member",
  "expiry": "2020-4-10 20:07:37",
  "expiry_time": 1586542057,
  "is_site_admin_view": false,
  "is_cluster_superuser": true
}


#### -3- Create a tensorFlow application cluster template for your tenant:

- Application templates are created from pre-configured, docker-based application images available out-of-the-box with HPE Container Platform **App Store** for big data, AI/ML tenants, and made visible by the IT administrator for your tenant (see picture below). These pre-configured images range from common Linux operating systems (CentOS, Ubuntu), Big Data and AI/ML frameworks, applications and tools such as open source distributions for Hadoop, Spark, TensorFlow and other frameworks, and utilities.   

- In HPE Container Platform, these frameworks and utilities generally refer to a distributed, single-node or multi-node application **virtual cluster**. Each application virtual cluster node runs as **a container** in the HPE Container Platform.

<img src="Pictures/HPECP-EPIC-tenant-App-Store-GUI.png" height="800" width="600" align="left">

The REST API call for creating an application cluster template is a POST request for a working tenant context (X-BDS-SESSION). The request (JSON) body specifies the application template name (20 characters max, **unique** among templates in the tenant) and a description in a label, as well as the application cluster specification (the number of nodes, their compute size and their roles, and the image in the catalog):

In [4]:
templateLocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/template" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
    "label": {
        "name": "'"$templateName"'", 
        "description": "TensorFlow with JupyterHub for '${username}'"
        },
    "clusterspec": {
        "cluster_type": "DataScience",
        "isolated": false,
        "dependent_nodegroups": [],
        "debug": false,
        "two_phase_delete": false,
        "nodegroup": {
            "role_configs": [
                {
                    "node_count": 1,
                    "flavor_name": "VeryTiny",
                    "role_id": "controller"
                },
                {
                    "node_count": 0,
                    "flavor_name": "VeryTiny",
                    "role_id": "jupyter"
                }
            ], 
            "catalog_entry_distro_id": "bluedata/tfjupyter-cpu",
            "config_choice_selections": [],
            "constraints": []
        }
    }
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your template Location: " $templateLocation
templateId=$(echo $templateLocation | cut -d'/' -f 5) # extract the template_id for later use
echo "And the template_id is: " $templateId

This is your template Location:  /api/v2/template/59
And the template_id is:  59


#### -4- Fetch the description of your template:

In [5]:
curl -k -s --request GET "https://${controller_endpoint}/api/v2/template/${templateId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' | jq 

{
  "_links": {
    "self": {
      "href": "/api/v2/template/59"
    },
    "all_templates": {
      "href": "/api/v2/template"
    }
  },
  "_embedded": {
    "label": {
      "_links": {
        "self": {
          "href": "/api/v2/template/59/label"
        }
      },
      "name": "ttfstudent80",
      "description": "TensorFlow with JupyterHub for student80"
    },
    "clusterspec": {
      "_links": {
        "self": {
          "href": "/api/v2/template/59/clusterspec"
        }
      },
      "cluster_type": "DataScience",
      "isolated": false,
      "dependent_nodegroups": [],
      "debug": false,
      "two_phase_delete": false,
      "nodegroup": {
        "role_configs": [
          {
            "node_count": 1,
            "flavor_name": "VeryTiny",
            "role_id": "controller"
          },
          {
            "node_count": 0,
            "flavor_name": "VeryTiny",
            "role_id": "jupyter"
          }
        ],
        "catalog_entry_distro_id": 

#### -5- Launch the application virtual cluster from the template:  
As the tenant user (with a role of admin or member), you can choose to launch an application virtual cluster using a programmatic approach based on the template you have just created. Let's create a TensorFlow application virtual cluster  using the template as starting point. As shown below, the JSON body specifies a cluster name (20 charcaters max) and description of the application virtual cluster as well as the resource location path for the created template object. The application cluster name must be **unique** among virtual clusters in the tenant. 

In [6]:
clusterLocation1=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/cluster" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
    "label":{
        "name":"'"$clusterName"'",
        "description":"TensorFlow Cluster for '${username}'"
        },
    "template":"'$templateLocation'"
    }'  | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your Cluster resource Location created from the template: " $clusterLocation1
clusterId1=$(echo $clusterLocation1 | cut -d'/' -f 5) # extract the cluster_id for later use
echo "And the cluster_id of your first app cluster is: " $clusterId1

This is your Cluster resource Location created from the template:  /api/v2/cluster/89
And the cluster_id of your first app cluster is:  89


#### -6- Verify the status of your virtual cluster:

In [7]:
curl -k -s --request GET "https://${controller_endpoint}/api/v2/cluster/${clusterId1}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' | jq  #using jq to pretty print the JSON reponse of the API call 

{
  "_links": {
    "self": {
      "href": "/api/v2/cluster/89"
    },
    "all_cluster_nodes": {
      "href": "/api/v2/cluster/89/node"
    },
    "all_clusters": {
      "href": "/api/v2/cluster"
    },
    "all_cluster_change_tasks": {
      "href": "/api/v2/cluster/89/change_task"
    },
    "all_cluster_change_histories": {
      "href": "/api/v2/cluster/89/change_history"
    },
    "all_cluster_action_histories": {
      "href": "/api/v2/cluster/89/action_history"
    },
    "all_cluster_action_tasks": {
      "href": "/api/v2/cluster/89/action_task"
    },
    "tenant": {
      "href": "/api/v1/tenant/3",
      "title": "HackShack Tenant"
    },
    "include_services": {
      "href": "/api/v2/cluster/89?services"
    },
    "include_nodes": {
      "href": "/api/v2/cluster/89?nodes"
    }
  },
  "label": {
    "name": "ctfstudent80",
    "description": "TensorFlow Cluster for student80"
  },
  "debug": false,
  "isolated": false,
  "two_phase_delete": false,
  "status": "sta

#### -7- Fetch the detailed information for the deployed virtual cluster nodes:

In [8]:
curl -k -s --request GET "https://${controller_endpoint}/api/v2/cluster/${clusterId1}?nodes" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' | jq

{
  "_links": {
    "self": {
      "href": "/api/v2/cluster/89?nodes"
    },
    "all_cluster_nodes": {
      "href": "/api/v2/cluster/89/node"
    },
    "all_clusters": {
      "href": "/api/v2/cluster"
    },
    "all_cluster_change_tasks": {
      "href": "/api/v2/cluster/89/change_task"
    },
    "all_cluster_change_histories": {
      "href": "/api/v2/cluster/89/change_history"
    },
    "all_cluster_action_histories": {
      "href": "/api/v2/cluster/89/action_history"
    },
    "all_cluster_action_tasks": {
      "href": "/api/v2/cluster/89/action_task"
    },
    "tenant": {
      "href": "/api/v1/tenant/3",
      "title": "HackShack Tenant"
    },
    "include_services": {
      "href": "/api/v2/cluster/89?services"
    }
  },
  "label": {
    "name": "ctfstudent80",
    "description": "TensorFlow Cluster for student80"
  },
  "debug": false,
  "isolated": false,
  "two_phase_delete": false,
  "status": "starting",
  "status_message": "0/1 booting",
  "log_url": "https://

#### -8- Get the service endpoints:  
Once you have identified the **cluster node Id** that is indicated inside the nodegroup section in the response body of your previous REST API call, fetch the services endpoints of the deployed application virtual cluster. You will need to pass the cluster node Id in the URI as well as the query parameter *services* to get endpoints networking information such as the HA proxy gateway host and ports (SSH, http/s). You will need this information to connect to the application you just deployed.

In [9]:
curl -k -s --request GET "https://${controller_endpoint}/api/v2/cluster/${clusterId1}/node/${nodes}?services" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' | jq

{
  "_links": {
    "self": {
      "href": "/api/v2/cluster/89/node?services"
    },
    "nodegroup_filter": {
      "href": "/api/v2/cluster/89/node?nodegroup={nodegroup}",
      "templated": true
    },
    "role_filter": {
      "href": "/api/v2/cluster/89/node?role={role}",
      "templated": true
    },
    "include_catalog_entry": {
      "href": "/api/v2/cluster/89/node?catalog_entry"
    },
    "include_flavor": {
      "href": "/api/v2/cluster/89/node?flavor"
    }
  },
  "_embedded": {
    "nodes": [
      {
        "_links": {
          "self": {
            "href": "/api/v2/cluster/89/node/107?services"
          },
          "cluster": {
            "href": "/api/v2/cluster/89",
            "title": "ctfstudent80"
          },
          "tenant": {
            "href": "/api/v1/tenant/3",
            "title": "HackShack Tenant"
          },
          "include_catalog_entry": {
            "href": "/api/v2/cluster/89/node/107?catalog_entry"
          },
          "include_f

As shown in the response of the REST API call, you can get information about the virtual cluster node : its status, role, the private service endpoints, and the public service endpoints.
You will use the public service endpoints to connect to your application services through the proxy host (aka Gateway) and proxy port (such as SSH or HTTP(s)). Access to application services running in containers is proxied via the gateway host and a port number greater than 10000:  
- SSH service: ssh -l your_login_username proxy_host -p proxy_port
- JupyterHub Service: https://proxy_host:proxy_port

#### -9- Check the application service is responding:
Identify the proxy_port for protocol HTTPS (a port number greater than 10000), and **replace** the port number in the command below:

In [10]:
curl -k -L -i -s https://gateway1.etc.fr.comm.hpecorp.net:10002 | grep "200 OK"

HTTP/1.1 200 OK


You can also open a browser session through our lab environment firewall: https://77.158.163.130:https_proxy_port where 77.158.163.130 is the NAT IP address of the HPE CP proxy gateway. You should then be able to login to the JupyterHub using your HPE CP tenant user's login credentials.

#### -10- Go through some cleanup: 
* First, delete the cluster:

In [11]:
curl -k -i -s --request DELETE "https://${controller_endpoint}/api/v2/cluster/${clusterId1}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json'

HTTP/1.1 202 Accepted
Access-Control-Allow-Origin: *
Content-Length: 0
Content-Type: text/plain
Date: Thu, 09 Apr 2020 18:11:00 GMT
Server: BlueData EPIC 5.0



The status *202 Accepted* means HPE Container Platform API server has accepted your request for deletion of the virtual cluster object.

* Then, delete the template:

In [12]:
curl -k -i -s --request DELETE "https://${controller_endpoint}/api/v2/template/${templateId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json'

HTTP/1.1 204 No Content
Access-Control-Allow-Origin: *
Content-Length: 0
Content-Type: text/plain
Date: Thu, 09 Apr 2020 18:11:03 GMT
Server: BlueData EPIC 5.0



The status *204 No Content* means the template object has been deleted.

* Although session have a time to live (TTL) of 24 hours, it is best practice in REST API programming to cleanup and delete those sessions when done. We can use a DELETE /api/v2/session/SessionId to achieve this:

In [13]:
curl -k -i -s --request DELETE "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json'

HTTP/1.1 204 No Content
Access-Control-Allow-Origin: *
Content-Length: 0
Content-Type: text/plain
Date: Thu, 09 Apr 2020 18:11:10 GMT
Server: BlueData EPIC 5.0



The status *204 No Content* means the session object has been deleted.

## Summary

In this tutorial, as a Big Data/AI (EPIC) tenant user, you have learned how to programmatically interact with HPE Container Platform to  create an application cluster template for a distributed application cluster image available in the App Store for your tenant. You have also experienced how to launch an instance of this cluster template for your tenant using the HPE Container Platform REST API, and connect to your deployed application.